In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TextStreamer
import torch
from openai import OpenAI
from IPython.display import display, update_display, Markdown, Image, Audio

In [3]:
import gradio as gr
import gc
import os

In [4]:
hf_token = userdata.get('HF_TOKEN_1')
login(hf_token, add_to_git_credential = True)

In [5]:
openai_api_key = userdata.get('HF_OPENAI')
openai= OpenAI(api_key = openai_api_key)

In [6]:
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [7]:
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_type = torch.bfloat16,
    bnb_4bit_quant_type = 'nf4'
)

In [8]:
system_prompt = """
You are an expert synthetic data generator AI.
Your task is to create realistic, structured datasets in JSON or CSV format
based purely on the user's description of their desired dataset.

Follow these principles strictly:

1. **Data Schema Inference**
   - Automatically infer meaningful column names, data types, and relationships from the user’s description.
   - Ensure column names use clear, consistent naming conventions (snake_case or camelCase).

2. **Data Realism**
   - Generate realistic, contextually appropriate values.
   - Use plausible ranges, types, and patterns (e.g., ages between 1–100, prices as decimals, valid dates, etc.).
   - If the domain involves categorical data (like gender, crop type, or sport), use relevant and diverse categories.

3. **Output Format Control**
   - Output **only the dataset**, without any explanations or extra commentary.
   - If the user requests `JSON`, output an array of JSON objects.
   - If the user requests `CSV`, output valid CSV with a header row.
   - Do not include markdown formatting (no triple backticks, etc.).

4. **Data Quantity**
   - Generate the requested number of rows.
   - Ensure that all rows are consistent in structure (no missing keys or inconsistent fields).

5. **Consistency and Cleanliness**
   - No nulls unless explicitly requested.
   - All numeric columns should be consistent in type (no mixing strings with numbers).
   - Avoid repetition — each row should feel unique and realistic.

6. **Field-Specific Intelligence**
   - For medical data: realistic names, patient IDs, diagnoses, ages, and treatment durations.
   - For agriculture: crops, rainfall, yield, fertilizer type, soil quality.
   - For education: students, subjects, scores, attendance, grades.
   - For sports: players, teams, scores, match dates, statistics.
   - For other domains: infer suitable fields based on context.

7. **Formatting Strictness**
   - JSON must be valid and parsable.
   - CSV must be comma-separated with consistent column order.

Example:

If the user says:
"Generate a dataset of 10 students with names, ages, and scores in mathematics."

You should output (for JSON):
[
  {"student_id": "S001", "name": "Amit Sharma", "age": 16, "math_score": 88},
  {"student_id": "S002", "name": "Priya Nair", "age": 15, "math_score": 92},
  ...
]

Or (for CSV):
student_id,name,age,math_score
S001,Amit Sharma,16,88
S002,Priya Nair,15,92
...

---

Remember:
- Never include explanations, notes, or commentary.
- The dataset **alone** should be the output.
- If the user gives a vague description, use your best inference and create useful columns relevant to that domain.
"""

In [9]:
def user_prompt(prompt):
    user_prompt = ""
    user_prompt += "You are an advanced AI specialized in generating realistic and structured synthetic datasets.\n\n"

    user_prompt += "User Description:\n"
    user_prompt += f"\"{prompt}\"\n\n"

    user_prompt += "Your objectives:\n"
    user_prompt += "1. **Schema Inference** – Automatically determine the most logical column names and data types based on the user's description.\n"
    user_prompt += "   - If the user specifies column names, follow them exactly.\n"
    user_prompt += "   - If not, intelligently infer suitable columns relevant to the context or domain.\n"
    user_prompt += "2. **Data Quantity** – By default, generate a moderately sized dataset (around 50–100 rows).\n"
    user_prompt += "   - Only change the number of rows if the user explicitly requests more or fewer.\n"
    user_prompt += "3. **Data Realism** – Ensure the data is contextually accurate, diverse, and coherent.\n"
    user_prompt += "   - Numeric values should stay within realistic ranges.\n"
    user_prompt += "   - Text or categorical values should reflect domain relevance (e.g., names, crops, schools, teams, etc.).\n"
    user_prompt += "4. **Structure & Consistency** – Every row must follow the same schema with no missing or mismatched fields.\n"
    user_prompt += "   - Maintain consistent data types across all rows.\n"
    user_prompt += "   - Avoid repetition — entries should look naturally varied.\n"
    user_prompt += "5. **Output Formatting** –\n"
    user_prompt += "   - If JSON is expected, output a single valid JSON array of objects.\n"
    user_prompt += "   - If CSV is expected, output a clean CSV with headers and properly aligned rows.\n"
    user_prompt += "   - Do not include markdown, explanations, or text outside the dataset.\n"
    user_prompt += "6. **Domain Awareness** – Adapt based on the topic:\n"
    user_prompt += "   - Medical → patients, age, diagnosis, treatment, recovery_days, etc.\n"
    user_prompt += "   - Agriculture → crop, fertilizer_type, rainfall, yield, soil_quality, etc.\n"
    user_prompt += "   - Education → student, subject, score, attendance, grade, etc.\n"
    user_prompt += "   - Sports → player, team, match_date, score, performance_stats, etc.\n"
    user_prompt += "   - If the domain is not clear, make a balanced general-purpose dataset.\n"
    user_prompt += "7. **Behavior** –\n"
    user_prompt += "   - Stay concise, structured, and realistic.\n"
    user_prompt += "   - Never include markdown, commentary, or explanations in your output.\n"
    user_prompt += "   - Output only the dataset — nothing else.\n"

    user_prompt += "\nYour response must contain ONLY the dataset output. No introductions, explanations, or summaries."

    return user_prompt


In [10]:
def messages_for(prompt):
  messages = [
      {'role':'system','content':system_prompt},
      {'role':'user','content':user_prompt(prompt)}
  ]

  return messages

In [11]:
def generate_dataset(model_name, prompt):
    messages = messages_for(prompt)
    tokenizer= AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer.apply_chat_template(messages_for(prompt),return_tensors = 'pt', add_generation_prompt = True).to('cuda')
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto', quantization_config = quant_config)
    output = model.generate(inputs, max_new_tokens = 5000)
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    del model, inputs, tokenizer, output
    gc.collect()
    torch.cuda.empty_cache()

    return result

## Saving the Dataset Generated

In [12]:
from google.colab import files
import time

In [13]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [14]:
SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/HF- Gen_AI/Datasets_Generated'
os.makedirs(SAVE_DIR, exist_ok=True)

In [17]:
def gen_and_save(model_name, prompt,file_format):
  result = generate_dataset(model_name, prompt)

  timestamp = int(time.time())
  filename = f"synthetic_dataset_{timestamp}.{file_format.lower()}"
  file_path = os.path.join(SAVE_DIR, filename)

  with open(file_path, "w", encoding="utf-8") as f:
        f.write(result)

  message = f"✅ Dataset successfully saved at:\n{file_path}"
  return result[:2500], message

## UI

In [16]:
gr.Interface(
    fn = gen_and_save,
    inputs = [gr.Dropdown([LLAMA], label = 'Select Your Model', value = LLAMA),gr.Textbox(label = 'Enter the Description', lines = 6),gr.Dropdown(['CSV','JSON'],label = 'Select the Result Format',value="CSV")],
    outputs = [gr.Markdown(label = "Result :"),gr.Markdown(label = 'Location :')],
    allow_flagging = 'never',
).launch(debug = True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2fb154767233ed0708.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2127, in proc

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2fb154767233ed0708.gradio.live
